In [1]:
import pandas as pd
import dask.dataframe as dd

/home/kchou/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
def parse_columns(df, cols):
    """Parses columns for a groupby-ed df"""
    cnt = df[cols]

    cnt = cnt.T
    cnt = cnt.reset_index(drop=True)
    cnt.columns = cnt.iloc[0]           # set first row to header
    cnt = cnt.drop(cnt.index[0])  # drop first row
    
    # create column for totals by sex
    if cols[1].endswith('Sex'):
        cnt.index =['Total']
    if cols[1].endswith('Male'):
        cnt.index =['Male']
    if cols[1].endswith('Female'):
        cnt.index =['Female']
    
    cnt.columns = range(len(cnt.columns))
    return cnt

In [3]:
def parse_group(df, dim):
    total = parse_columns(df, [dim, 'Dim: Sex (3): Member ID: [1]: Total - Sex'])
    male = parse_columns(df, [dim, 'Dim: Sex (3): Member ID: [2]: Male'])
    female = parse_columns(df, [dim, 'Dim: Sex (3): Member ID: [3]: Female'])

    unioned = pd.concat([total, male, female])
    return unioned

In [4]:
def parse_census_profile(df, dask=False):
    cols = df.columns
    
    # Get item to groupby on
    level = [s for s in df.columns if s.startswith('DIM:')][0] 
    #count_dim = [s for s in df.columns if s.startswith('Dim:')]
    
    # Map integers to column names
    column_map = {i:v for i,v in enumerate(df.loc[df.GEO_NAME == 'Canada', level].tolist())}
    
    if dask:
        df = dd.from_pandas(df, npartitions=20)
        parsed_df = df.groupby(['GEO_CODE (POR)', 'GEO_NAME']).apply(parse_group, level, meta={i:object for i in range(2247)}).compute()
    else:
        parsed_df = df.groupby(['GEO_CODE (POR)', 'GEO_NAME']).apply(parse_group, level)
        
    parsed_df = parsed_df.rename(columns=column_map)
    
    return parsed_df

# FED

In [5]:
df = pd.read_csv("98-401-X2016045_English_CSV_data.csv", low_memory=False)

In [6]:
fed = parse_census_profile(df)

In [7]:
fed2 = parse_census_profile(df, dask=True)

### Census Tract

In [8]:
ct = pd.read_csv('98-401-X2016043_English_CSV_data.csv', low_memory=False)

In [9]:
ctp = parse_census_profile(ct, dask=True)

# dissemination area

In [5]:
import os

In [6]:
files_dir = 'da'
files = [f'{files_dir}/' + f for f in os.listdir(files_dir)]

In [7]:
files

['da/98-401-X2016044_ATLANTIC_English_CSV_data.csv',
 'da/98-401-X2016044_TERRITORIES_English_CSV_data.csv']

In [15]:
df = pd.read_csv(files[0])

/home/kchou/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
%time da_t = parse_census_profile(df, dask=True)

CPU times: user 2min 43s, sys: 1min 41s, total: 4min 24s
Wall time: 2min 47s


In [17]:
%time da_t2 = parse_census_profile(df, dask=False)

CPU times: user 1min 36s, sys: 11.3 s, total: 1min 47s
Wall time: 1min 45s


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 24.6 µs


In [ ]:

1